# Classificação de textos através de Modelos de Linguagem baseados na arquitetura transformers


* Neste caso usaremos um dataset divido em textos ofensivos e não ofensivos e aplicaremos o BERTimbau

In [ ]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


Importações

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


Carregando dataset e tokenizador,utilizaremos o bertimbau.

In [ ]:
dataset = load_dataset("ruanchaves/hatebr")

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4480 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1120 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1400 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Pré-processamento

In [4]:
def preprocess_function(examples):
    return tokenizer(
        examples["instagram_comments"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
# Tokenizar o dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["instagram_comments"])
# Renomear a coluna de rótulo principal
if 'labels' not in tokenized_datasets['train'].column_names:
    tokenized_datasets = tokenized_datasets.rename_column("offensive_language", "labels")

Map:   0%|          | 0/4480 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Dado as limitações de memória ram e o elevado custo computacional,optei por utilizar treinamento em batch

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select([i for i in range(1000)])
small_val_dataset = tokenized_datasets["validation"].shuffle(seed=42).select([i for i in range(200)])


In [6]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'labels': torch.tensor([item['labels'] for item in batch], dtype=torch.float)  # Converter para float
    }


In [7]:
train_dataloader = DataLoader(small_train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(small_val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)


Carregando o modelo

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=1) #num_labels=1 p/classificação binária
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Configurando otimizador e scheduler

In [9]:
# Configurar otimizador e scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * 5  # 5 épocas
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

Função de perda,neste caso para classificação binária


In [10]:
criterion = torch.nn.BCEWithLogitsLoss()


Definindo funções para treinamento e para avaliar o modelo com fine-tunning

In [11]:
# Função de treinamento
def train():
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).unsqueeze(1)  # Ajustar a forma dos labels para [batch_size, 1]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = criterion(logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average Training Loss: {avg_train_loss:.4f}")

In [12]:
# Função de avaliação
def evaluate():
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).unsqueeze(1)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.round(torch.sigmoid(logits))  # Converter logits em previsões binárias

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


Realizando treinamento de fato enquanto avalia-o por época

In [ ]:
for epoch in range(10):
    train()
    metrics = evaluate()
    print(f"Epoch {epoch + 1}: {metrics}")


Average Training Loss: 0.4870
Epoch 1: {'accuracy': 0.86, 'precision': 0.9176470588235294, 'recall': 0.7878787878787878, 'f1': 0.8478260869565216}
Average Training Loss: 0.1858
Epoch 2: {'accuracy': 0.915, 'precision': 0.9183673469387755, 'recall': 0.9090909090909091, 'f1': 0.9137055837563451}
Average Training Loss: 0.0695
Epoch 3: {'accuracy': 0.895, 'precision': 0.9431818181818182, 'recall': 0.8383838383838383, 'f1': 0.8877005347593583}
Average Training Loss: 0.0342
Epoch 4: {'accuracy': 0.905, 'precision': 0.9081632653061225, 'recall': 0.898989898989899, 'f1': 0.9035532994923858}
Average Training Loss: 0.0238
Epoch 5: {'accuracy': 0.895, 'precision': 0.90625, 'recall': 0.8787878787878788, 'f1': 0.8923076923076922}
Average Training Loss: 0.0221
Epoch 6: {'accuracy': 0.895, 'precision': 0.90625, 'recall': 0.8787878787878788, 'f1': 0.8923076923076922}
